# Daily Portfolio Values

Here are the steps to calculate the daily portfolio values:

1. Create a dataframe of prices with columns for each stock, indexed by date
2. Normalize the prices dataframe (prices / prices[0]) into a normalized dataframe. These represent cumulative returns starting from the start date.
3. Multiply the normalized dataframe by the allocations for each stock into an allocations dataframe.
4. Multipy the allocations dataframe by the start value (how much money you started with) to calculate the amount of cash allocated to each stock. Save this into a postions dataframe.
5. Calculate the portfolio values each day by summing the positions across the `1` axis. Save this into a portfolio_value dataframe (or a series in your existing dataframe)

In [1]:
import pandas as pd
from util import get_data
from util import plot_data

In [4]:
dates = pd.date_range('2012-01-01', '2012-01-10')
symbols = ['SPY', 'GLD', 'XOM', 'GOOG']
df = get_data(symbols, dates)
print df

               SPY     GLD    XOM    GOOG
2012-01-03  126.29  155.92  84.40  665.41
2012-01-04  126.49  156.71  84.42  668.28
2012-01-05  126.82  157.78  84.16  659.01
2012-01-06  126.50  157.20  83.53  650.02
2012-01-09  126.80  156.50  83.91  622.46
2012-01-10  127.90  158.64  84.12  623.14


In [7]:
prices_df = df / df.iloc[0]
print prices_df

                 SPY       GLD       XOM      GOOG
2012-01-03  1.000000  1.000000  1.000000  1.000000
2012-01-04  1.001584  1.005067  1.000237  1.004313
2012-01-05  1.004197  1.011929  0.997156  0.990382
2012-01-06  1.001663  1.008209  0.989692  0.976871
2012-01-09  1.004038  1.003720  0.994194  0.935453
2012-01-10  1.012748  1.017445  0.996682  0.936475


In [9]:
allocations_df = prices_df * [0.4, 0.4, 0.1, 0.1]
print allocations_df

                 SPY       GLD       XOM      GOOG
2012-01-03  0.400000  0.400000  0.100000  0.100000
2012-01-04  0.400633  0.402027  0.100024  0.100431
2012-01-05  0.401679  0.404772  0.099716  0.099038
2012-01-06  0.400665  0.403284  0.098969  0.097687
2012-01-09  0.401615  0.401488  0.099419  0.093545
2012-01-10  0.405099  0.406978  0.099668  0.093648


In [11]:
start_value = 1000000
positions_df = allocations_df * start_value
print positions_df

                      SPY            GLD            XOM           GOOG
2012-01-03  400000.000000  400000.000000  100000.000000  100000.000000
2012-01-04  400633.462665  402026.680349  100023.696682  100431.313025
2012-01-05  401678.676063  404771.677783   99715.639810   99038.186982
2012-01-06  400665.135799  403283.735249   98969.194313   97687.140259
2012-01-09  401615.329797  401487.942535   99419.431280   93545.332953
2012-01-10  405099.374456  406977.937404   99668.246445   93647.525586


In [14]:
portfolio_value_series = positions_df.sum(axis=1)
print portfolio_value_series

2012-01-03    1000000.000000
2012-01-04    1003115.152722
2012-01-05    1005204.180639
2012-01-06    1000605.205620
2012-01-09     996068.036563
2012-01-10    1005393.083891
dtype: float64


# Portfolio Statistics

In [16]:
def compute_daily_returns(df):
    daily_returns = (df / df.shift(1)) - 1
    daily_returns.iloc[0, :] = 0 # Replace NaN with 0
    return daily_returns

df2 = pd.DataFrame(portfolio_value_series, columns=['portfolio_value'])
compute_daily_returns(df2)

,portfolio_value
2012-01-03,0.000000
2012-01-04,0.003115
2012-01-05,0.002083
2012-01-06,-0.004575
2012-01-09,-0.004534
2012-01-10,0.009362


Note: I'm not sure the above computation for computing daily returns is correct.

# Computing Sharpe Ratio

`sharpe = mean(daily returns - daily risk free rate of return) / std(daily returns - daily risk free rate of return)`

What is the risk free rate of return? Some possible sources:
- LIBOR (London Inter-Bank Offered Rate)
- 3 month T-Bill
- 0%

Shortcut for calculating the risk-free rate:

`s = mean(daily returns - daily rf) / std(daily returns)`

# Quiz: What Is The Sharpe Ratio?

Given:
- 60 days of data
- Averate Daily Return = 0.001
- Daily Risk Free Rate = 0.0002
- Standard Deviation Return = 0.001

`k * mean(_daily_rets_ - _daily_rf_) / std(_daily_rets_)`

`k = sqrt(252)`

```
>>> math.sqrt(252) * (0.001 - 0.0002) / 0.001
12.699606293110037
```